# Model Utilities

> Functions Used In Modeling Efforts

In [ ]:
#| default_exp modeling.utils

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export

from data_system_utilities.azure.storage import FileHandling
from machine_learning_utilities import preprocessing

from LTBP.data.utils import snowflake_query, get_yaml_dicts

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from rfpimp import *

import sklearn.preprocessing as y_transform
import os
import logging
import pickle

In [ ]:
#| export


def return_dict_type(
    pre_process_type:dict # {k:v} dictionary of columns name and tranformation type
                    ):
    """
    Simplify the standard process for sklearn preprocessing pipelines
    """
    for k, v in pre_process_type.items():
        if v == "OrdinalEncoder":
            pre_process_dict = {
                f"{k}": {
                    "transformation": {
                        "name": "OrdinalEncoder",
                        "args": {
                            "handle_unknown": "use_encoded_value",
                            "unknown_value": -1,
                        },
                    },
                    "variable_type": "cat",
                }
            }
        if v == "OneHotEncoder":
            pre_process_dict = {
                f"{k}": {
                    "transformation": {
                        "name": "OneHotEncoder",
                        "args": {"handle_unknown": "ignore", "sparse": False},
                    },
                    "variable_type": "cat",
                }
            }
        if v == "StandardScaler":
            pre_process_dict = {
                f"{k}": {
                    "transformation": {"name": "StandardScaler", "args": {}},
                    "variable_type": "cont",
                }
            }
        if v == "RobustScaler":
            pre_process_dict = {
                f"{k}": {
                    "transformation": {"name": "RobustScaler", "args": {}},
                    "variable_type": "cont",
                }
            }
    return pre_process_dict


In [ ]:
from data_system_utilities.file_parsers import yaml

In [ ]:
feature_dict=yaml.yaml_reader('./LTBP/files/yaml_files/features.yaml')

cat_vars =[{f.lower() : values['transformation']} for f, values in feature_dict.items() 
            if values['var_type'] == 'cat'
            and values['input_definition'] != 'LABEL']

return_dict_type(cat_vars[0])

{'destinationgeoafinitylabel': {'transformation': {'name': 'OrdinalEncoder',
   'args': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1}},
  'variable_type': 'cat'}}

In [ ]:
#| hide
test_eq(return_dict_type(cat_vars[0]).keys(), cat_vars[0].keys())
test_eq(return_dict_type(cat_vars[0])[list(cat_vars[0].keys())[0]].keys(), ['transformation', 'variable_type'])

In [ ]:
#| export


def create_sklearn_preprocess_baseline_dict(
    cat_vars:list, # list of categorical variables with sklearn transformer
    cont_vars:list, # list of continous variables with sklearn transformer
):
    """wrapper around ``return_dict_type`` to go through cat and cont vars
    """
    final_dict = {}
    if cat_vars is None:
        cat_vars = []
    if cont_vars is None:
        cont_vars = []
    for item in cat_vars + cont_vars:
        final_dict.update(return_dict_type(item))
    return final_dict


In [ ]:
feature_dict=yaml.yaml_reader('./LTBP/files/yaml_files/features.yaml')

cat_vars =[{f.lower() : values['transformation']} for f, values in feature_dict.items() 
            if values['var_type'] == 'cat'
            and values['input_definition'] != 'LABEL']
cont_vars =[{f.lower(): values['transformation']} for f, values in feature_dict.items() 
            if values['var_type'] == 'cont'
            and values['input_definition'] != 'LABEL']

feature_dict = create_sklearn_preprocess_baseline_dict(cat_vars=cat_vars, 
                                                       cont_vars=cont_vars)
logging.info(feature_dict)

INFO:root:{'destinationgeoafinitylabel': {'transformation': {'name': 'OrdinalEncoder', 'args': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1}}, 'variable_type': 'cat'}, 'evercorepass': {'transformation': {'name': 'OrdinalEncoder', 'args': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1}}, 'variable_type': 'cat'}, 'everpass': {'transformation': {'name': 'OrdinalEncoder', 'args': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1}}, 'variable_type': 'cat'}, 'gendercode': {'transformation': {'name': 'OrdinalEncoder', 'args': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1}}, 'variable_type': 'cat'}, 'guestbehavior': {'transformation': {'name': 'OrdinalEncoder', 'args': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1}}, 'variable_type': 'cat'}, 'isepicmixactivated': {'transformation': {'name': 'OrdinalEncoder', 'args': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1}}, 'variable_type': 'cat'}, 'marketingzone': {'transfo

In [ ]:
#| hide
test_eq(feature_dict[list(cat_vars[0].keys())[0]].keys(), ['transformation', 'variable_type'])

In [ ]:
#| export


def return_list_of_vars(variables):
    """_summary_

    Args:
        variables (_type_): _description_

    Returns:
        _type_: _description_
    """
    if variables is None:
        return None
    vars_list = []
    for item in variables:
        for k in item.keys():
            vars_list.append(k)
    return vars_list


In [ ]:
cat_vars = return_list_of_vars(cat_vars)
cont_vars = return_list_of_vars(cont_vars)
logging.info(f'categorical variables: \n {cat_vars}')
logging.info(f'continous variables: \n {cont_vars}')

INFO:root:categorical variables: 
 ['destinationgeoafinitylabel', 'evercorepass', 'everpass', 'gendercode', 'guestbehavior', 'isepicmixactivated', 'marketingzone', 'mostcommonticketcomp', 'mostsubseasonvisited', 'mostvisitedregion', 'mostvisitedresort', 'onlysingleresortkey', 'partnerresortscannerflag', 'skierabilitylabel', 'totalseasonsscanned', 'visitmostinpeak']
INFO:root:continous variables: 
 ['age', 'avgvisitperseason', 'resortsvisited', 'subseasonsperyear', 'totalvisits']


In [ ]:
#| hide
feature_dict=yaml.yaml_reader('./LTBP/files/yaml_files/features.yaml')
test_eq(cat_vars, [f.lower() for f, values in feature_dict.items()  
                   if values['var_type'] == 'cat' 
                   and values['input_definition'] != 'LABEL'])
test_eq(cont_vars, [f.lower() for f, values in feature_dict.items() 
                   if values['var_type'] == 'cont' 
                   and values['input_definition'] != 'LABEL'])

In [ ]:
#| export


def prepare_training_set(df,
                         y_var,
                         sklearn_pipe,
                         model_dict: dict,
                         connection_str: str,
                         model_grain: str,
                         scaler_type,
                         test_set=True):
    """split and preprocess data set"""
    if scaler_type == 'MinMaxScaler':
        scaler = MinMaxScaler()
    else:
        # TODO: use sklearn get attr
        # will always expect a sklearn like api
        scaler = scaler_type

    # Sklearn basic split method
    X_train, X_valid, y_train, y_valid = train_test_split(df, df[y_var], test_size=.20, random_state=1320)
    if test_set is True:
        X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=.15, random_state=1320)
        logging.info(f'Successfully Spilt Data\nTrain: {X_train.shape}, {y_train.shape}\nValid: {X_valid.shape}, {y_valid.shape}\nTest: {X_test.shape}, {y_test.shape}')
    else:
        y_test = None
        X_test = None
        logging.info(f'Successfully Spilt Data\nTrain: {X_train.shape}, {y_train.shape}\nValid: {X_valid.shape}, {y_valid.shape}')
    y_train = scaler.fit_transform(y_train.reset_index()[[y_var]])
    y_train = pd.DataFrame(y_train)
    y_train.columns = [y_var]
    y_valid = scaler.transform(y_valid.reset_index()[[y_var]])
    y_valid = pd.DataFrame(y_valid)
    y_valid.columns = [y_var]
    if test_set is True:
        y_test = scaler.transform(y_test.reset_index()[[y_var]])
        y_test = pd.DataFrame(y_test)
        y_test.columns = [y_var]
    save_sklearn_object_to_data_lake(save_object=scaler,
                                     file_name=model_dict['y_preprocess_object_name'],
                                     adls_path=os.path.join(os.environ['CI_COMMIT_SHA'],
                                                            model_dict['preprocessors_adls_path'], model_grain),
                                     container_name=model_dict['azure_container'],
                                     connection_str=connection_str)
    X_train = sklearn_pipe.fit_transform(X_train)
    cols = preprocessing.get_column_names_from_transformer(sklearn_pipe)
    X_train = pd.DataFrame(X_train)
    X_train.columns = cols

    X_valid = sklearn_pipe.transform(X_valid)
    cols = preprocessing.get_column_names_from_transformer(sklearn_pipe)
    X_valid = pd.DataFrame(X_valid)
    X_valid.columns = cols
    if test_set is True:
        X_test = sklearn_pipe.transform(X_test)
        cols = preprocessing.get_column_names_from_transformer(sklearn_pipe)
        X_test = pd.DataFrame(X_test)
        X_test.columns = cols
    save_sklearn_object_to_data_lake(save_object=sklearn_pipe,
                                     file_name=model_dict['x_preprocess_object_name'],
                                     adls_path=os.path.join(os.environ['CI_COMMIT_SHA'],
                                                            model_dict['preprocessors_adls_path'], model_grain),
                                     container_name=model_dict['azure_container'],
                                     connection_str=connection_str)
    return X_train, X_valid, X_test, y_train, y_valid, y_test, sklearn_pipe, scaler

``preprocess_data`` 

This one needs a test, but as of right now holding off on this test as this test would take a little big more time than I want to spend on the documentation of this process at the current moment

The idea of this function is to ensure that the user is using the pre-processor in the correct fashion so that the validation set is not being considered in the pre-processing dictionary

In [ ]:
#| export


def preprocess_data(X_train, X_valid, X_test, sklearn_pipe):
    X_train = sklearn_pipe.fit_transform(X_train)
    cols = preprocessing.get_column_names_from_transformer(sklearn_pipe)
    X_train = pd.DataFrame(X_train)
    X_train.columns = cols

    X_valid = sklearn_pipe.transform(X_valid)
    cols = preprocessing.get_column_names_from_transformer(sklearn_pipe)
    X_valid = pd.DataFrame(X_valid)
    X_valid.columns = cols

    X_test = sklearn_pipe.transform(X_test)
    cols = preprocessing.get_column_names_from_transformer(sklearn_pipe)
    X_test = pd.DataFrame(X_test)
    X_test.columns = cols

    return X_train, X_valid, X_test, sklearn_pipe

``save_sklearn_object_to_data_lake``

This function is simply wrapping DSU functionality together to allow for a model to be pushed to adls these tests are written and evaluated inside of DSU

In [ ]:
#| export


def save_sklearn_object_to_data_lake(
    save_object, file_name, adls_path, container_name, connection_str
):
    """moves a sklearn object to azure data lake as a pickle file at a given path"""
    logging.info(
        f"Pushing Sklearn Object to Azure: {os.path.join(adls_path, file_name)}"
    )
    with open(file_name, "wb") as f:
        pickle.dump(save_object, f)
    az = FileHandling(connection_str)
    az.upload_file(
        azure_file_path=adls_path,
        local_file_path=file_name,
        container_name=container_name,
        overwrite=True,
    )
    os.unlink(file_name)
    logging.info(f"{file_name} successfully pushed to {adls_path}")

In [ ]:
#| export


def evaluate(model, X_valid, y_valid, y_var, feature_importance=True):
    """
    Utlity to give experiment table information about the model
    this is fully customizable and can be changed to be regression
    RMSE, R2, MSE for example and changing the columns this function
    isn't a dynamic function it needs to be written for a specific use
    case.

    Args:
    * model (classifer): sklearn model for this
    * X_valid (np.array): Validation set Traing
    * y_valid (np.array): Actuals for Validation
    * y_var (str): variable name being predicted

    Returns:
    * dict: dependent on return statement
    """
    y_pred_proba = model.predict_proba(X_valid)
    y_pred = model.predict(X_valid)
    auc = metrics.roc_auc_score(y_valid, y_pred_proba[:, 1])
    acc = metrics.accuracy_score(y_valid, y_pred)
    bacc = metrics.balanced_accuracy_score(y_valid, y_pred)
    columns = ['auc', 'acc', 'bacc']
    logging.info(f'{y_var} AUC: {auc:.3f}    Accuracy: {acc:.3f}    Balanced Accuracy: {bacc:.3f}')
    if feature_importance is True:
        feat_df = pd.read_csv('features.csv')
        logging.info(f'features: /n {feat_df.columns}')
        feat_df = dict(zip(feat_df.dropna()['featurenames'], feat_df.dropna()['dtypes']))
        logging.info(f'reduced to independent variables: /n {feat_df}')
        fi_permutation = importances(model, pd.DataFrame(X_valid, columns=list(feat_df.keys())), pd.DataFrame(y_valid)) # noqa:
        fi_permutation = (fi_permutation
                          .reset_index()
                          .rename({'Feature': 'cols', 'Importance': 'imp'}, axis=1))
        logging.info(f'Feature Importance df: \n {fi_permutation}')
        fi_permutation.to_csv('fi_permutation.csv', index=False)
    return auc, acc, bacc, columns

In [ ]:
#| export


def evaluate_test(model, X_test, y_test, y_var, ecid_list, table_name,
                  grain_name=None, grain_value=None, container_name='vailadls',
                  blob_dest='./', connection_str=os.environ['DATALAKE_CONN_STR_SECRET'],
                  file_output='./holdout.csv', sfSchema='DEV'):
    """
    Utlity to give us the scores on a complete test hold out

    Args:
    * sf (sf_connection): snowflake connection
    * model (classifer): sklearn model for this
    * X_valid (np.array): Validation set Traing
    * y_valid (np.array): Actuals for Validation
    * y_var (str): variable name being predicted
    * ecid_list (list): customer id list with season year
    """
    y_pred_proba = model.predict_proba(X_test)
    y_pred = model.predict(X_test)
    auc = metrics.roc_auc_score(y_test, y_pred_proba[:, 1])
    acc = metrics.accuracy_score(y_test, y_pred)
    bacc = metrics.balanced_accuracy_score(y_test, y_pred)
    logging.info(f'For Test Set Unseen Data {y_var} AUC: {auc:.3f} Accuracy: {acc:.3f} Balanced Accuracy: {bacc:.3f}')
    logging.info(f'Size of Test Data set: {len(y_test)}')
    test_df_results = pd.DataFrame(ecid_list)
    if grain_value is not None:
        test_df_results['SUBSEASON'] = grain_value
    test_df_results['PROBABILITY'] = y_pred_proba[:, 1]
    logging.info(f'Preview Test Push df \n {test_df_results.head(2)}')
    logging.info(f'Preview Test Size: \n {test_df_results.shape[0]}')
    sf = snowflake_query(sfSchema=sfSchema)
    sf.infer_to_snowflake(test_df_results,
                          table_name=table_name)
    logging.info('saving test prediction file')
    test_df_results.to_csv(file_output, index=False)
    logging.info(f'sending prediction file to azure {container_name} to {blob_dest}')
    # blob_pusher(file_path=[file_output],
    #             container_name=container_name,
    #             blob_dest=[blob_dest],
    #             connection_str=connection_str,
    #             overwrite=True)
    # os.unlink(file_output)

In [ ]:
#| export


def move_dev_holdout_table_to_prod_location(sf, exp):
    logging.info('Replacing Prod HoldOut With Newest Promoted')
    sf.run_str_query(f"""
                      CREATE OR REPLACE TABLE MACHINELEARNINGOUTPUTS.ltbp.{exp['holdout_tb_name']} AS
                      SELECT * FROM MACHINELEARNINGOUTPUTS.DEV.{exp['holdout_tb_name']};
                      """)

In [ ]:
#| export


def holdout_set_to_prod(connection_str: str, yaml_files: list = ['etl.yaml', 'experiment.yaml'], sfschema: str = 'ltbp'):
    sf = snowflake_query(sfSchema=sfschema)
    etl, exp = get_yaml_dicts(yaml_files)
    fh = FileHandling(connection_str)
    if exp['holdout_tb_name'].upper() in sf.run_str_query("show tables;").name.tolist():
        logging.info(f"Deleting current prod hold out {exp['holdout_tb_name'].upper()}")
        sf.run_str_query(f"DROP TABLE MACHINELEARNINGOUTPUTS.{sfschema}.{exp['holdout_tb_name']};")
    logging.info("inference only pipeline pushing hold out data set to prodction table")
    # timestamp = query_production_timestamp(exp, etl)
    path = [etl['data_lake']['stage_path'].replace('COMMITID', "TODO:") + '/holdout/']
    ls_list = fh.ls_blob(container_name='vailadls',
                         path=path[0],
                         recursive=True)
    for p in [path[0] + x for x in ls_list]:
        logging.warning('if the model and data are not in alphabetical order this method will not work will need to customize')
        logging.info(f'holdout data set {p[0]}')
        # blob_puller(files=[p],
        #             connection_str=connection_str,
        #             container_name='vailadls',
        #             drop_location='.',
        #             overwrite=True
        #             )
        df = pd.read_csv(p.split('/')[-1])
        logging.info(f'Preview Test \n {df.head(2)}')
        logging.info(f'Preview Test Size: \n {df.shape[0]}')
        sf.infer_to_snowflake(df,
                              table_name=exp['holdout_tb_name'].upper())

Will generate a test for this at a later time as LTBP doesn't need this type of massaging



The y_var inside of LTBP isn't a great use case, but RVF where we might want to scale the y_var in regression or time series using MinMaxScaler or StandardScaler this will give the flexibility needed

In [ ]:
#| skip


def y_var_scaler(y_train, y_valid, y_test, y_var, scaler_type):
    """Write Doc String"""
    y_train = scaler_type.fit_transform(y_train.reset_index()[[y_var]])
    y_train = pd.DataFrame(y_train)
    y_train.columns = [y_var]
    y_valid = scaler_type.transform(y_valid.reset_index()[[y_var]])
    y_valid = pd.DataFrame(y_valid)
    y_valid.columns = [y_var]
    y_test = scaler_type.transform(y_test.reset_index()[[y_var]])
    y_test = pd.DataFrame(y_test)
    y_test.columns = [y_var]
    return y_train, y_valid, y_test, scaler

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()